L'objectif de ce script est de mesurer les performances d'une stratégie breakout 

In [1]:
import requests
import datetime
import pandas as pd
import numpy as np
import time

In [12]:
# --------- VARIABLES ---------
# 'WIFUSDT', '1000BONKUSDT', 'SANDUSDT'
coin = 'ETHUSDT'
date_debut = '2020-08-21 00:00:00'
#date_fin = datetime.datetime.now()
date_fin = '2021-04-21 00:00:00'
date_fin = pd.to_datetime(date_fin)


# BYBIT

def get_info():
    url = 'https://api.bybit.com/v5/market/instruments-info?category={}'.format('linear')
    data = requests.get(url).json()
    print(data)


def get_history_bybit(ticker, start, limit=1000):
    start = int(datetime.datetime.timestamp(pd.to_datetime(start))*1000)
    end_date = start + 5731200000
    url = 'https://api.bybit.com/v5/market/kline?category={}&symbol={}&limit={}&interval={}&end={}'.format('linear', ticker, limit, 60, end_date)
    columns = ['time', 'open', 'high', 'low', 'close', 'volume', 'turnover']
    data = requests.get(url).json()
    df = pd.DataFrame(data['result']['list'], columns=columns)
    df.index = [pd.to_datetime(x, unit='ms').strftime('%Y-%m-%d %H:%M:%S') for x in df.time]
    #df = df[columns]
    df = df.sort_values('time')
    return df

def get_latest_data_bybit(ticker):
    df = pd.DataFrame()
    start_date = pd.to_datetime(date_debut)  # Convertir en objet datetime

    while True:
        new_data = get_history_bybit(ticker, start=start_date.strftime('%Y-%m-%d %H:%M:%S'))
        if not start_date.date() >= date_fin.date():
            df = pd.concat([df, new_data])            
        else:
            break
        start_date += datetime.timedelta(milliseconds=5731200000)
        time.sleep(1)  # Pause pour éviter de surcharger l'API

    df = df.drop_duplicates()
    #print(df)
    return df


df = get_latest_data_bybit(coin)
print(df)

(0, 0)
(0, 7)
(0, 7)


C:\Users\b.bellity_ext\AppData\Local\Temp\ipykernel_13244\4075837540.py:25: FutureWarning: The behavior of 'to_datetime' with 'unit' when parsing strings is deprecated. In a future version, strings will be parsed as datetime strings, matching the behavior without a 'unit'. To retain the old behavior, explicitly cast ints or floats to numeric type before calling to_datetime.
  df.index = [pd.to_datetime(x, unit='ms').strftime('%Y-%m-%d %H:%M:%S') for x in df.time]


(0, 7)
(1000, 7)
                              time     open     high      low    close  \
2021-04-01 16:00:20  1617292800000  1943.55   1956.5  1939.25   1947.2   
2021-04-01 16:59:19  1617296400000   1947.2  1965.95   1930.7   1940.8   
2021-04-01 18:00:29  1617300000000   1940.8     1980  1884.55  1969.15   
2021-04-01 18:59:28  1617303600000  1969.15   1988.4  1963.35     1977   
2021-04-01 20:00:38  1617307200000     1977   1986.1   1968.1   1972.1   
...                            ...      ...      ...      ...      ...   
2021-05-13 02:59:16  1620874800000  3986.95  3988.05  3863.05   3947.4   
2021-05-13 04:00:26  1620878400000   3947.4     3982  3891.25  3908.15   
2021-05-13 04:59:25  1620882000000  3908.15  4017.85  3908.15   3988.4   
2021-05-13 06:00:35  1620885600000   3988.4     4025  3970.75   4019.3   
2021-05-13 06:59:34  1620889200000   4019.3   4023.7  3947.25     3970   

                       volume       turnover  
2021-04-01 16:00:20   5015.83    9766824.176  


C:\Users\b.bellity_ext\AppData\Local\Temp\ipykernel_13244\4075837540.py:25: FutureWarning: The behavior of 'to_datetime' with 'unit' when parsing strings is deprecated. In a future version, strings will be parsed as datetime strings, matching the behavior without a 'unit'. To retain the old behavior, explicitly cast ints or floats to numeric type before calling to_datetime.
  df.index = [pd.to_datetime(x, unit='ms').strftime('%Y-%m-%d %H:%M:%S') for x in df.time]


In [11]:
print(df.shape)

(1000, 7)


In [3]:
# Assurez-vous que les colonnes sont de type float
df['high'] = pd.to_numeric(df['high'], errors='coerce')
df['low'] = pd.to_numeric(df['low'], errors='coerce')
df['open'] = pd.to_numeric(df['open'], errors='coerce')
df['close'] = pd.to_numeric(df['close'], errors='coerce')

# Calcul des maximums sur 24h et 7 jours glissants
df['24h_high'] = df['high'].shift(1).rolling(window=24).max()
df['breakout_24h'] = df['high'] > df['24h_high'] 
df['closed_above_breakout_24h'] = df['close'] > df['24h_high']

df['7d_high'] = df['high'].shift(1).rolling(window=7*24).max()   
df['breakout_7d'] = df['high'] > df['7d_high']
df['closed_above_breakout_7d'] = df['close'] > df['7d_high']



In [4]:
# --------------------  CALCUL PERFORMANCE --------------------

# PERFORMANCE FROM BREAKOUT -------------------------------

# Calcul de la performance après un breakout sur 24h
df['perf_break24h_h1'] = np.where(df['breakout_24h'], (df['close'].shift(-1) - df['24h_high']) / df['24h_high'], np.nan)
df['perf_break24h_h4'] = np.where(df['breakout_24h'], (df['close'].shift(-4) - df['24h_high']) / df['24h_high'], np.nan)

# Calcul de la performance après un breakout sur 7d
df['perf_break7d_h1'] = np.where(df['breakout_7d'], (df['close'].shift(-1) - df['7d_high']) / df['7d_high'], np.nan)
df['perf_break7d_h4'] = np.where(df['breakout_7d'], (df['close'].shift(-4) - df['7d_high']) / df['7d_high'], np.nan)
df['perf_break7d_h12'] = np.where(df['breakout_7d'], (df['close'].shift(-12) - df['7d_high']) / df['7d_high'], np.nan)
df['perf_break7d_h24'] = np.where(df['breakout_7d'], (df['close'].shift(-24) - df['7d_high']) / df['7d_high'], np.nan)

# PERFORMANCE FROM CLOSE  -------------------------------

# Calcul de la performance après un breakout sur 24h
df['perf_break24h_h1_from_close'] = np.where(df['breakout_24h'], (df['close'].shift(-1) - df['close']) / df['close'], np.nan)
df['perf_break24h_h4_from_close'] = np.where(df['breakout_24h'], (df['close'].shift(-4) - df['close']) / df['close'], np.nan)

# Calcul de la performance après un breakout sur 7d
df['perf_break7d_h1_from_close'] = np.where(df['breakout_7d'], (df['close'].shift(-1) - df['close']) / df['close'], np.nan)
df['perf_break7d_h4_from_close'] = np.where(df['breakout_7d'], (df['close'].shift(-4) - df['close']) / df['close'], np.nan)
df['perf_break7d_h12_from_close'] = np.where(df['breakout_7d'], (df['close'].shift(-12) - df['close']) / df['close'], np.nan)
df['perf_break7d_h24_from_close'] = np.where(df['breakout_7d'], (df['close'].shift(-24) - df['close']) / df['close'], np.nan)

# DRAWDOWN MAX FROM OPEN  -------------------------------

df['drawdown_max_h1'] = (df['low'] - df['open']) / df['open']
df['drawdown_max_h4'] = (df['low'].rolling(window=4).min() - df['open']) / df['open']
df['drawdown_max_h12'] = (df['low'].rolling(window=12).min() - df['open']) / df['open']
df['drawdown_max_h24'] = (df['low'].rolling(window=24).min() - df['open']) / df['open']



In [5]:
print(df['drawdown_max_h1'])

2021-04-01 16:00:20   -0.002212
2021-04-01 16:59:19   -0.008474
2021-04-01 18:00:29   -0.028983
2021-04-01 18:59:28   -0.002945
2021-04-01 20:00:38   -0.004502
                         ...   
2021-05-13 02:59:16   -0.031076
2021-05-13 04:00:26   -0.014225
2021-05-13 04:59:25    0.000000
2021-05-13 06:00:35   -0.004425
2021-05-13 06:59:34   -0.017926
Name: drawdown_max_h1, Length: 1000, dtype: float64


In [6]:
filtered_df = df[df['breakout_24h'] == True].head(30)
print(filtered_df)

                              time     open     high      low    close  \
2021-04-02 20:00:14  1617393600000  2062.75  2109.90  2062.35  2092.70   
2021-04-02 22:00:23  1617400800000  2091.70  2117.10  2088.85  2111.30   
2021-04-02 22:59:22  1617404400000  2111.30  2147.00  2111.30  2136.65   
2021-04-05 14:00:48  1617631200000  2081.10  2128.15  2081.10  2121.15   
2021-04-05 19:59:04  1617652800000  2107.30  2134.95  2101.40  2108.05   
2021-04-05 23:59:22  1617667200000  2107.60  2148.65  2107.60  2145.35   
2021-04-06 01:00:32  1617670800000  2145.35  2157.50  2122.10  2125.80   
2021-04-08 13:00:38  1617886800000  2029.65  2040.10  2024.35  2029.40   
2021-04-08 13:59:37  1617890400000  2029.40  2053.00  2029.35  2043.55   
2021-04-08 15:00:47  1617894000000  2043.55  2061.65  2042.85  2061.15   
2021-04-08 15:59:46  1617897600000  2061.15  2078.25  2061.15  2068.85   
2021-04-08 22:59:12  1617922800000  2071.35  2085.25  2071.35  2081.70   
2021-04-09 00:00:22  1617926400000  20

In [8]:
file_path = './data/breakout/' + coin + '.csv'
df.to_csv(file_path , index=True)